In [1]:
from collections import Counter
from dateutil import parser
from datetime import timezone
from pprint import pprint

import ipywidgets as ipyw
import traitlets as trt
from IPython.display import display

import requests

import ipyelk
import ipyelk.nx
from ipyelk.diagram.elk_model import ElkLabel

In [2]:
try:
    import sysml_v2_api_client as sysml2
    from sysml_v2_api_client.rest import ApiException
except ImportError:
    !pip install git+https://github.com/Systems-Modeling/SysML-v2-API-Python-Client.git
    print(
        "Had to install the SysML v2 Python API Client.\n"
        "Restart the kernel and run the notebook again..."
    )

In [3]:
class SysML2Client(ipyw.VBox):

    host_url = trt.Unicode(
        default_value="http://sysml2-sst.intercax.com"
    )
    host_port = trt.Integer(
        default_value=9000,
        min=1,
        max=65535,
    )
    
    page_size = trt.Integer(
        default_value=2000,
        min=10,
    )

    _api_configuration = trt.Instance(sysml2.Configuration)
    _commits_api = trt.Instance(sysml2.CommitApi)
    _elements_api = trt.Instance(sysml2.ElementApi)
    _projects_api = trt.Instance(sysml2.ProjectApi)

    projects = trt.Dict()
    elements_by_id = trt.Dict()
    elements_by_type = trt.Dict()
    
    host_url_input = trt.Instance(ipyw.Text)
    host_port_input = trt.Instance(ipyw.IntText)
    project_selector = trt.Instance(ipyw.Dropdown)
    commit_selector = trt.Instance(ipyw.Dropdown)
    download_elements = trt.Instance(ipyw.Button)
    type_selector = trt.Instance(ipyw.SelectMultiple, kw=dict())
    
    @trt.validate("children")
    def _validate_children(self, proposal):
        children = proposal.value
        if children:
            return children
        return [
            ipyw.HTML("<h2>Client</h2>"),
            ipyw.HBox([
                ipyw.VBox([
                    ipyw.HBox([self.host_url_input, self.host_port_input]),
                    self.project_selector,
                    self.commit_selector,
                ]),
                self.download_elements,
            ]),
            ipyw.HTML("<h2>Element Types<h2>"),
            self.type_selector,
        ]

    @trt.default("_api_configuration")
    def _make_api_configuration(self):
        return sysml2.Configuration(host=self.host)

    @trt.default("_commits_api")
    def _make_commits_api(self):
        with sysml2.ApiClient(self._api_configuration) as client:
            api = sysml2.CommitApi(client)
        return api

    @trt.default("_elements_api")
    def _make_elements_api(self):
        with sysml2.ApiClient(self._api_configuration) as client:
            api = sysml2.ElementApi(client)
        return api

    @trt.default("_projects_api")
    def _make_projects_api(self):
        with sysml2.ApiClient(self._api_configuration) as client:
            api = sysml2.ProjectApi(client)
        return api

    @trt.default("projects")
    def _make_projects(self):
        projects = self._projects_api.get_projects()
        return {
            project.id: dict(
                created=parser.parse(
                    " ".join(project.name.split()[-6:])
                ).astimezone(timezone.utc),
                full_name=project.name,
                name=" ".join(project.name.split()[:-6]),
            )
            for project in projects
        }
    
    @trt.default("host_url_input")
    def _make_host_url_input(self):
        input_box = ipyw.Text(default_value=self.host_url)
        trt.link(
            (self, "host_url"),
            (input_box, "value"),
        )
        input_box.layout.width = "80%"
        return input_box
    
    @trt.default("host_port_input")
    def _make_host_port_input(self):
        input_box = ipyw.IntText(
            default_value=self.host_port,
            min=1,
            max=65535,
        )
        trt.link(
            (self, "host_port"),
            (input_box, "value"),
        )
        input_box.layout.width = "15%"
        return input_box
    
    @trt.default("project_selector")
    def _make_project_selector(self):
        selector = ipyw.Dropdown(
            descriptor="Projects",
            options=self._get_project_options(),
        )
        selector.observe(self._update_commit_options, "value")
        selector.layout.width = "95%"
        return selector
    
    @trt.default("commit_selector")
    def _make_commit_selector(self):
        selector = ipyw.Dropdown(
            descriptor="Commits",
            options=self._get_commit_options(),
        )
        selector.observe(self._update_elements, "value")
        selector.layout.width = "95%"
        return selector
    
    @trt.default("download_elements")
    def _make_download_elements_button(self):
        button = ipyw.Button(
            icon="download",
            tooltip="Download elements",
        )
        button.on_click(self._download_elements)
        button.layout.height = "90%"
        return button

    @trt.observe("host_url", "host_port")
    def _update_api_configuration(self, *_):
        self._api_configuration = self._make_api_configuration()
    
    @trt.observe("_api_configuration")
    def _update_apis(self, *_):
        for api_type in ("commit", "element", "project"):
            api_attr = f"_{api_type}s_api"
            old_api = getattr(self, api_attr)
            api_maker = getattr(self, f"_make{api_attr}")
            setattr(self, api_attr, api_maker())
            del old_api
        self.project_selector.options = self._get_project_options()
        
    @property
    def host(self):
        return f"{self.host_url}:{self.host_port}"

    @property
    def selected_project_id(self):
        return self.project_selector.value

    @property
    def selected_commit_id(self):
        return self.commit_selector.value

    @property
    def elements_url(self):
        return (
            f"{self.host}/"
            f"projects/{self.selected_project_id}/"
            f"commits/{self.selected_commit_id}/"
            f"elements?page[size]={self.page_size}"
        )
    
    @property
    def selected_elements_by_type(self):
        element_ids = sum(map(list, self.type_selector.value), [])
        return [
            self.elements_by_id[id_]
            for id_ in element_ids
        ]

    def _update_commit_options(self, *_):
        self.commit_selector.options = self._get_commit_options()

    def _update_elements(self, *_):
        pass

    def _get_project_options(self):
        project_name_instances = Counter(
            project["name"]
            for project in self.projects.values()
        )

        return {
            data["name"] + (
                f""" ({data["created"].strftime("%Y-%m-%d %H:%M:%S")})"""
                if project_name_instances[data["name"]] > 1
                else ""
            ): id_
            for id_, data in sorted(
                self.projects.items(),
                key=lambda x: x[1]["name"],
            )
        }

    def _get_commit_options(self):
        # TODO: add more info about the commit when API provides it
        return [
            commit.id
            for commit in self._commits_api.get_commits_by_project(
                self.selected_project_id
            )
        ]

    def _download_elements(self, *_):
        response = requests.get(self.elements_url)
        if not response.ok:
            raise requests.HTTPError(
                f"Failed to retrieve elements from '{self.elements_url}', "
                f"reason: {response.reason}"
            )
        elements = response.json()
        self.elements_by_id = {
            element["@id"]: element
            for element in elements
        }
        self.elements_by_type = {
            type_: tuple([
                el["@id"]
                for el in elements
                if el["@type"] == type_
            ])
            for type_ in set(element["@type"] for element in elements)
        }
        
    def by_id(id_: str):
        return self.elements_by_id[id_]

    def name_by_id(id_: str):
        return self.by_id(id_).get("Name")
    
    @trt.observe("elements_by_type")
    def _updated_type_selector_options(self, *_):
        self.type_selector.options = {
            f"{type_} [{len(elements)}]": elements
            for type_, elements in sorted(self.elements_by_type.items())
        }

In [4]:
client = SysML2Client()
client

SysML2Client(children=(HTML(value='<h2>Client</h2>'), HBox(children=(VBox(children=(HBox(children=(Text(value=…

In [5]:
client.project_selector.value = "d2922960-028d-4534-a8ee-476aa26d1d25"
client._download_elements()

In [7]:
client.selected_elements_by_type

[{'@context': {'@base': 'http://sysml2-sst.intercax.com:9000/projects/d2922960-028d-4534-a8ee-476aa26d1d25/commits/61cb843b-b6ad-4868-9fb8-29094e59827e/elements/',
   '@vocab': 'http://omg.org/ns/sysml#',
   'sysml': 'http://omg.org/ns/sysml#',
   'dcterms': 'http://purl.org/dc/terms/',
   'xsd': 'http://www.w3.org/2001/XMLSchema#',
   'aliasId': {'@type': 'xsd:string'},
   'documentation': {'@type': '@id'},
   'documentationComment': {'@type': '@id'},
   'general': {'@type': '@id'},
   'humanId': {'@type': 'xsd:string'},
   'identifier': {'@type': 'dcterms:identifier'},
   'name': {'@type': 'xsd:string'},
   'ownedAnnotation': {'@type': '@id'},
   'ownedElement': {'@type': '@id'},
   'ownedRelatedElement': {'@type': '@id'},
   'ownedRelationship': {'@type': '@id'},
   'ownedTextualRepresentation': {'@type': '@id'},
   'owner': {'@type': '@id'},
   'owningClassifier': {'@type': '@id'},
   'owningMembership': {'@type': '@id'},
   'owningNamespace': {'@type': '@id'},
   'owningRelatedEle